In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import os
import cv2
from tqdm import tqdm
from utils import get_2D_projections
from datetime import datetime
import traceback

%env SITK_SHOW_COMMAND 'C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'

env: SITK_SHOW_COMMAND='C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'


In [2]:
import sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from Utils import utils

In [3]:
config = utils.read_config()
config

[{'paths': {'source_path_wd': 'E:/U-CAN-Lymfom_A',
   'source_path_bd': 'E:/ucan_lymfom'}},
 {'filenames': {'incomplete_folders_filename': 'No_PTorCT_exams_from_U-CAN-Lymfom.xlsx',
   'final_selected_images_filename': 'FinalSelected_exams_from_U-CAN-Lymfom.xlsx',
   'list_of_distorted_images_filename': 'distorted_lst.txt'}},
 {'filenames': {'final_selected_images_filename': 'Finalized_dataset.xlsx'}},
 {'resampling': {'voxel_spacing': [2, 2, 3], 'image_size': [384, 384, 384]}},
 {'paths': {'destination_path': 'Selected_for_UCAN_project/2dprojections'},
  'projections': {'modality': ['CT', 'PET', 'SUV'],
   'tissue_type': ['LT', 'AT', 'LT', 'B', 'N'],
   'projection_type': ['sum', 'min', 'max', 'mean', 'std']}}]

In [4]:
config[0]['paths']

{'source_path_wd': 'E:/U-CAN-Lymfom_A', 'source_path_bd': 'E:/ucan_lymfom'}

In [5]:
source_path_wd = config[0]['paths']['source_path_wd']
source_path_bd = config[0]['paths']['source_path_bd']
final_selected_images_filename = config[2]['filenames']['final_selected_images_filename']
projection_path = config[4]['paths']['destination_path']

selection_dataframe = os.path.join(source_path_wd, final_selected_images_filename)
final_projections = os.path.join(source_path_wd, projection_path)
final_projections

'E:/U-CAN-Lymfom_A\\Selected_for_UCAN_project/2dprojections'

In [6]:
selection_dataframe

'E:/U-CAN-Lymfom_A\\Finalized_dataset.xlsx'

In [7]:
master_df = pd.read_excel(selection_dataframe)
try:
    drop_col_list = [col for col in master_df.columns if 'unnamed' in col.lower()]
    master_df.drop(columns=drop_col_list, inplace=True)
except:
    pass
print(master_df.shape)
master_df.head(2)

(1316, 42)


,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,dicom_img,patient_id,patient_age,...,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,ASPTCTX0001_npr134758508773-20191212,CT-20191212-111939-3.7-WB_FDG-CT-VENFAS-WB-Ven...,ASPTCTX0001,npr134758508773,20191212,1.2.840.113619.2.405.3.3842664208.103.15760743...,npr134758508773,74.0,...,CT,'GE MEDICAL SYSTEMS','Discovery MI',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,ASPTCTX0001_npr134758508773-20191212,PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S...,ASPTCTX0001,npr134758508773,20191212,1.2.840.113619.2.405.3842664208.1576145387.106224,npr134758508773,74.0,...,PT,'GE MEDICAL SYSTEMS','Discovery MI','FDG -- fluorodeoxyglucose','0.0','095300.00','194729152.0','6586.2001953125','0.96700000762939','20191212095300.00'


In [8]:
config[3]

{'resampling': {'voxel_spacing': [2, 2, 3], 'image_size': [384, 384, 384]}}

In [9]:
new_size = config[3]['resampling']['image_size']
new_spacing = config[3]['resampling']['voxel_spacing']
print('new_size: ', new_size)
print('new_spacing: ', new_spacing)


new_size:  [384, 384, 384]
new_spacing:  [2, 2, 3]


In [10]:
projections = config[4]['projections']
projections

{'modality': ['CT', 'PET', 'SUV'],
 'tissue_type': ['LT', 'AT', 'LT', 'B', 'N'],
 'projection_type': ['sum', 'min', 'max', 'mean', 'std']}

In [11]:
modality = projections['modality']
type = projections['tissue_type']
ptype = projections['projection_type']

In [12]:
exception_lst = []
if not os.path.exists(final_projections):
    os.mkdir(final_projections)

for index, row in master_df.iterrows():
    if row['modality']=='PT':
        #vol_img = sitk.ReadImage(row['directory'])
        
        print(row['directory'])

        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(row['directory'])
        reader.SetFileNames(dicom_names)
        vol_img = reader.Execute() 

        vol_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                            vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                            vol_img.GetPixelID())
        
        save_path = os.path.join(final_projections, str(row['patient_directory']) + '_PET_resample_3Dimg')
        
        utils.get_2D_projections(vol_img, modality[1], ptype=ptype[2], angle=45, img_n=save_path)
        
        suv, estimated, raw,spacing,origin,direction = utils.compute_suv(vol_img, PatientWeight=row['patient_weight'], AcquisitionTime=str(row['aquisition_dt'])+str(row['aquisition_time']) , RadiopharmaceuticalStartTime=row['radiopharmaceutical_start_time'], RadionuclideHalfLife=row['radionuclide_half_life'], RadionuclideTotalDose=row['radionuclide_total_dose'])
        suv_img = sitk.GetImageFromArray(suv)
        
        save_path= os.path.join(final_projections, str(row['patient_directory']) + '_SUV_resample_3Dimg')

        utils.get_2D_projections(suv_img, modality[2], ptype=ptype[2], angle=45, img_n=save_path)
        break

E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr134758508773-20191212/PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_
161977.3616731149 0.0
56.12243533553609 0.0


v_img = sitk.ReadImage(os.path.join(source_path_wd, 'Ashish_SampleResampling/pat1_PET_resample_3Dimg.nii.gz'))
if not os.path.exists(os.path.join(source_path_wd, 'Ashish_SampleResampling/2d projections')):
    os.mkdir(os.path.join(source_path_wd, 'Ashish_SampleResampling/2d projections'))

save_path = os.path.join(final_projections, 'pat1_PET_resample_3Dimg')

#get_2D_projections(v_img,MODA,PTYPE,15,T_TYPE, img_n=save_path)
get_2D_projections(v_img,modality[1],ptype=ptype[2],angle=15,img_n=save_path)
save_path= os.path.join(final_projections, '/CT_sample3Dimg')

get_2D_projections(v_img,modality[1], type[0],ptype[2],15,img_n=save_path)